In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
import json
import os
import requests
import re
import pickle
from nltk import ngrams
from fuzzymatchlist import FuzzyList

### Notebook for finding and cleaning Business names and entities.

#### Logic:

* Check the 'Address Book Type'
    * If Business Enttity: Run cleaning logic on "Name" field.
    * If Individual/Candidate Family: Run cleaning logic on "Employer Name" field.
* Create a dictionary of 

In [3]:
transaction_detail_df = pd.read_csv("../scrape_files/transaction_detail_first_batch.csv", low_memory=False)

In [4]:
with open("state_city.pickle", 'rb') as f:
    state_city_dict = pickle.load(f)

In [5]:
for col in transaction_detail_df.columns:
    print(col)

Address
Address Book Type
Agent
Aggregate
Amount
Associations
Check
CoSigner Obligations
Description
Due Date
Employer Name
Exam Letter Date
Filed Date
In-Kind/Independent Expenditures
Interest Rate
Name
Occupation
Occupation Letter Date
Payer of Personal Expenditure
Payment Method
Process Status
Purpose
Repayment Schedule
Transaction Date
Transaction ID
Transaction Sub Type
Transaction Type


In [6]:
np.unique(list(transaction_detail_df["Address Book Type"]))

array(['Business Entity', "Candidate's Immediate Family", 'Individual',
       'Labor Organization', 'Other', 'Political Committee',
       'Political Party Committee', 'Unregistered Committee', 'nan'],
      dtype='<U28')

In [7]:
business_sub_df = transaction_detail_df[transaction_detail_df["Address Book Type"] == "Business Entity"].copy()
individual_sub_df = transaction_detail_df[transaction_detail_df["Address Book Type"] == "Individual"].copy()

In [126]:
def get_name_parts(employer_name):
    city = None
    state = employer_name[-2:]
    remainder = employer_name[:-2]
    deep = 3
    n_gram_list = []
    for n in range(1,deep+1):
        current = [set([str.join(" ", x)]) for x in list(ngrams(remainder.strip().split(" ")[-deep:],n))]
        n_gram_list.append(current)
    try:
        pos_cities = state_city_dict[state]
    except:
        return "Key Error"
    for ng_list in n_gram_list:
        for ng_set in ng_list:
            if len(pos_cities.intersection(ng_set)) > 0:
                city = list(ng_set)[0]
    cleaned_city = re.sub("({})".format(city), "", get_backwards_string(employer_name), 1)
    cleaned_employer = re.sub("({})".format(state), "", cleaned_city, 1)
    return get_backwards_string(cleaned_employer.strip()), city, state

In [127]:
def get_backwards_string(string):
    return str.join(" ", string.split(" ")[::-1])

In [130]:
for employer_name in list(individual_sub_df["Employer Name"]):
    try:
        employer_name_l = employer_name.lower()
    except:
        print(employer_name_l)
        continue
    if employer_name_l in ["self employed", "not employed"]:
        print(employer_name)
    else:
        print(get_name_parts(employer_name_l), employer_name)

Not Employed
Not Employed
('centennial school district 28j', 'portland', 'or') Centennial School District 28J Portland OR
Self Employed
('city of portland, oregon', 'portland', 'or') City of Portland, Oregon Portland OR
('centennial school district', 'portland', 'or') Centennial School District Portland OR
Not Employed
('centennial school district', 'portland', 'or') Centennial School District Portland OR
Self Employed
Self Employed
Self Employed
Self Employed
Self Employed
('gerding edlen', 'portland', 'or') Gerding Edlen Portland OR
('city commissioner sam adams', 'portland', 'or') City Commissioner Sam Adams Portland OR
Self Employed
('centennial school district', 'portland', 'or') Centennial School District Portland OR
('centennial school district', 'portland', 'or') Centennial School District Portland OR
('macadam forbes', 'portland', 'or') Macadam Forbes Portland OR
Self Employed
Self Employed
('centennial school district', 'portland', 'or') Centennial School District Portland OR

Not Employed
('native american rehabilitation assn', 'portland', 'or') Native American Rehabilitation Assn Portland OR
('schwabe williamson & wyatt pc', 'portland', 'or') Schwabe Williamson & Wyatt PC Portland OR
('strategic solutions nw', 'beaverton', 'or') Strategic Solutions NW Beaverton OR
('gallatin group', 'portland', 'or') Gallatin Group Portland OR
Self Employed
('rentrak', 'portland', 'or') Rentrak Portland OR
('gallatin public affairs', 'portland', 'or') Gallatin Public Affairs Portland OR
Self Employed
Self Employed
('city of portland', 'portland', 'or') City of Portland Portland OR
('landye bennett blumstein llp', 'portland', 'or') Landye Bennett Blumstein LLP Portland OR
('campaign for equal justice', 'portland', 'or') Campaign for Equal Justice Portland OR
Self Employed
self employed
('atrium capital menlo park', 'menlo park', 'ca') Atrium Capital Menlo Park CA
Self Employed
('providence', 'portland', 'or') Providence Portland OR
Not Employed
('williams & dame development

('northwest strategies inc.', 'portland', 'or') Northwest Strategies Inc. Portland OR
Not Employed
('ohsu', 'portland', 'or') OHSU Portland OR
Self Employed
('polycycle industrial products, inc. clackamas', None, 'or') Polycycle Industrial Products, Inc. Clackamas OR
('friends of martha schrader', 'milwaukie', 'or') Friends of Martha Schrader Milwaukie OR
Self Employed
Not Employed
('norwest engineering', 'portland', 'or') Norwest Engineering Portland OR
('gard communications', 'portland', 'or') Gard Communications Portland OR
Self Employed
('carroll community investments llc', 'portland', 'or') Carroll Community Investments LLC Portland OR
('kaiser permanente', 'portland', 'or') Kaiser Permanente Portland OR
('town center bank', 'milwaukie', 'or') Town Center Bank Milwaukie OR
Self Employed
Not Employed
('stoll berne', 'portland', 'or') Stoll Berne Portland OR
('boutin & associates lake oswego', 'lake oswego', 'or') Boutin & Associates Lake Oswego OR
('advanced technology group', 'hil

Not Employed
Not Employed
Not Employed
('timberline lodge government camp', 'government camp', 'or') Timberline Lodge Government Camp OR
Self Employed
('metro', 'portland', 'or') Metro Portland OR
Self Employed
Not Employed
Not Employed
Not Employed
Self Employed
('clackamas county oregon city', 'oregon city', 'or') Clackamas County Oregon City OR
clackamas county oregon city or
Self Employed
('schwabe, williamson & wyatt', 'portland', 'or') Schwabe, Williamson & Wyatt Portland OR
('produce row prop mgt co', 'portland', 'or') Produce Row Prop Mgt Co Portland OR
('the yoshida group', 'portland', 'or') The Yoshida Group Portland OR
('dark horse comics, inc.', 'milwaukie', 'or') Dark Horse Comics, Inc. Milwaukie OR
('portland general electric', 'portland', 'or') Portland General Electric Portland OR
portland general electric portland or
Not Employed
('audubon society of portland', 'portland', 'or') Audubon Society of Portland Portland OR
audubon society of portland portland or
Not Employe

('providence health and services', 'portland', 'or') Providence Health and Services Portland OR
Self Employed
Not Employed
('providence health & services', 'portland', 'or') Providence Health & Services Portland OR
('cascade aids project', 'portland', 'or') Cascade AIDS Project Portland OR
Not Employed
Not Employed
Not Employed
Not Employed
Not Employed
('museum at warm  springs', 'warm springs', 'or') Museum at Warm Springs Warm Springs OR
('museum at warm  springs', 'warm springs', 'or') Museum at Warm Springs Warm Springs OR
('clifton law', 'bend', 'or') Clifton Law Bend OR
('mazamas', 'portland', 'or') Mazamas Portland OR
('mckinley irvin', 'portland', 'or') McKinley Irvin Portland OR
Not Employed
('the flame', 'portland', 'or') The Flame Portland OR
Not Employed
Not Employed
Not Employed
('deloitte san francisco', 'san francisco', 'ca') Deloitte San Francisco CA
Not Employed
('lauder partners llc', 'atherton', 'ca') Lauder Partners LLC atherton CA
Not Employed
('womencount palo al

...

('teamsters local no. 305', 'portland', 'or') Teamsters Local No. 305 Portland OR
('teamsters local no. 305', 'portland', 'or') Teamsters Local No. 305 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local 162', 'portland', 'or') Teamsters Local 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local no. 162', 'portland', 'or') Teamsters Local No. 162 Portland OR
('teamsters local union 

('josephine county grants pass', 'grants pass', 'or') Josephine County Grants Pass OR
Self Employed
('sofcu grants pass', 'grants pass', 'or') SOFCU Grants Pass OR
Not Employed
Key Error State of Oregon Salem Oregon
Self Employed
Not Employed
('state of oregon', 'salem', 'or') State of Oregon Salem OR
Not Employed
Not Employed
Self Employed
Not Employed
Not Employed
Not Employed
Self Employed
('stein interprises grants pass', 'grants pass', 'or') Stein Interprises Grants Pass OR
Self Employed
Not Employed
Self Employed
('state of oregon', 'salem', 'or') State of Oregon Salem OR
Self Employed
Not Employed
('sofcu grants pass', 'grants pass', 'or') SOFCU Grants Pass OR
('state of oregon', 'salem', 'or') State of Oregon Salem OR
Not Employed
Not Employed
Self Employed
('state of oregon', 'salem', 'or') State of Oregon Salem OR
Not Employed
('josephine county grants pass', 'grants pass', 'or') Josephine County Grants Pass OR
('state of oregon', 'salem', 'or') State of Oregon Salem OR
state

('city of portland', 'portland', 'or') City of Portland Portland OR
('portland police bureau', 'portland', 'or') Portland Police Bureau Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland, port.police assn. vigil', 'portland', 'or') City of Portland, Port.Police Assn. Vigil Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portland Portland OR
('city of portland', 'portland', 'or') City of Portlan

('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') City of Eugene Eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') City of Eugene Eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') City of Eugene Eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') City of Eugene Eugene OR
('eugene of eugene', 'eugene', 'or') eugene of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') c

('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') City of Eugene Eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city of eugene eugene OR
('city of eugene', 'eugene', 'or') city 